In [1]:
import numpy as np
from sklearn.datasets import load_boston

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
dataset = load_boston()
X = dataset['data']
y = dataset['target']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [5]:
x_train_a, x_train_b, y_train_a, y_train_b = train_test_split(x_train, y_train)

In [6]:
x_train_a.shape

(284, 13)

In [8]:
gs_ridge = RidgeCV(alphas=np.logspace(-2, 2))

In [9]:
gs_ridge.fit(x_train_a, y_train_a)

RidgeCV(alphas=array([  1.00000e-02,   1.20679e-02,   1.45635e-02,   1.75751e-02,
         2.12095e-02,   2.55955e-02,   3.08884e-02,   3.72759e-02,
         4.49843e-02,   5.42868e-02,   6.55129e-02,   7.90604e-02,
         9.54095e-02,   1.15140e-01,   1.38950e-01,   1.67683e-01,
         2.02359e-01,   2...     3.90694e+01,   4.71487e+01,   5.68987e+01,   6.86649e+01,
         8.28643e+01,   1.00000e+02]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [10]:
meta1 = gs_ridge.predict(x_train_b)

In [11]:
# pervyy priznak dlia kompozicii
meta1.shape

(95,)

In [12]:
gcv_knn = GridSearchCV(KNeighborsRegressor(), {'n_neighbors':np.arange(1, 20)})
gcv_knn.fit(x_train_a, y_train_a)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [13]:
gcv_knn.best_params_

{'n_neighbors': 2}

In [14]:
meta2 = gcv_knn.predict(x_train_b)

In [15]:
meta2.shape

(95,)

In [17]:
meta1[0], meta2[0]

(24.724431027760897, 22.699999999999999)

In [18]:
gcv_rf = GridSearchCV(RandomForestRegressor(n_estimators=30, n_jobs=-1), {'max_depth':np.arange(2, 15)})
gcv_rf.fit(x_train_a, y_train_a)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [20]:
meta3 = gcv_rf.predict(x_train_b)

In [21]:
from sklearn.metrics import mean_absolute_error

In [24]:
print('knn', mean_absolute_error(y_test, gcv_knn.predict(x_test)))
print('rf', mean_absolute_error(y_test, gcv_rf.predict(x_test)))
print('ridge', mean_absolute_error(y_test, gs_ridge.predict(x_test)))

knn 4.53031496063
rf 2.68594841642
ridge 3.51490227917


In [25]:
new_x_train = np.vstack((meta1, meta2, meta3)).T

In [26]:
meta_ridge = RidgeCV(np.logspace(-2, 2))
meta_ridge.fit(new_x_train, y_train_b)

new_x_test = np.vstack((
            gs_ridge.predict(x_test),
            gcv_knn.predict(x_test),
            gcv_rf.predict(x_test))).T
y_pred = meta_ridge.predict(new_x_test)

In [27]:
mean_absolute_error(y_test, y_pred)

2.4242185595834336

In [28]:
from sklearn.preprocessing import PolynomialFeatures

In [29]:
# postroit poparnuye kombinacii vsex priznakov
pf = PolynomialFeatures(include_bias=False)
pf.fit(new_x_train)
pf_new_x_train = pf.transform(new_x_train)
pf_new_x_test = pf.transform(new_x_test)

In [30]:
meta_ridge = RidgeCV(np.logspace(-2, 2))
meta_ridge.fit(pf_new_x_train, y_train_b)
y_pred = meta_ridge.predict(pf_new_x_test)
mean_absolute_error(y_test, y_pred)


2.4805818809759277